<a href="https://colab.research.google.com/github/kodenshacho/sigma/blob/master/glfragment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://suzulang.com/usage-glactivetexture/

https://heinleinsgame.tistory.com/m/9

https://github.com/jiangjie8/OpenGLSimple/tree/master/3rdparty/Windows

https://168iroha.net/blog/topic/?id=202008111602&sorting=post_date

In [ ]:

#include <GL/glew.h>
#include <GL/freeglut.h>
#include <iostream>
#include <vector>
#define STB_IMAGE_IMPLEMENTATION
#include "stb_image.h"

const char* imagePaths[] = {
    "image1.jpg",
    "image2.jpg",
    "image3.jpg"
};

GLuint textures[3];
GLuint volumeTexture;

void loadTexture(int index) {
    int width, height, channels;
    unsigned char* image = stbi_load(imagePaths[index], &width, &height, &channels, 0);
    if (!image) {
        std::cerr << "Failed to load texture: " << imagePaths[index] << std::endl;
        return;
    }

    glGenTextures(1, &textures[index]);
    glBindTexture(GL_TEXTURE_2D, textures[index]);
    // 텍스처 파라미터 설정
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
    glTexParameteri(GL_TEXTURE_2D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);

    glTexImage2D(GL_TEXTURE_2D, 0, GL_RGB, width, height, 0, GL_RGB, GL_UNSIGNED_BYTE, image);
    glGenerateMipmap(GL_TEXTURE_2D);
    stbi_image_free(image);
}

void loadVolumeTexture() {
    const int volumeSize = 64;
    std::vector<unsigned char> volumeData(volumeSize * volumeSize * volumeSize, 0);

    // 仮想データ
    for (int z = 0; z < volumeSize; ++z) {
        for (int y = 0; y < volumeSize; ++y) {
            for (int x = 0; x < volumeSize; ++x) {
                volumeData[z * volumeSize * volumeSize + y * volumeSize + x] = 128;
            }
        }
    }

    glGenTextures(1, &volumeTexture);
    glBindTexture(GL_TEXTURE_3D, volumeTexture);

    glTexParameteri(GL_TEXTURE_3D, GL_TEXTURE_WRAP_S, GL_CLAMP_TO_EDGE);
    glTexParameteri(GL_TEXTURE_3D, GL_TEXTURE_WRAP_T, GL_CLAMP_TO_EDGE);
    glTexParameteri(GL_TEXTURE_3D, GL_TEXTURE_WRAP_R, GL_CLAMP_TO_EDGE);
    glTexParameteri(GL_TEXTURE_3D, GL_TEXTURE_MIN_FILTER, GL_LINEAR);
    glTexParameteri(GL_TEXTURE_3D, GL_TEXTURE_MAG_FILTER, GL_LINEAR);

    glTexImage3D(GL_TEXTURE_3D, 0, GL_RED, volumeSize, volumeSize, volumeSize, 0, GL_RED, GL_UNSIGNED_BYTE, volumeData.data());
    glGenerateMipmap(GL_TEXTURE_3D);
}


const int windowWidth = 800;
const int windowHeight = 600;

// カメラ位置
glm::vec3 cameraPos = glm::vec3(0.0f, 0.0f, 3.0f);

// ボリュームレンダリング用のシェーダーコード
const char* volumeVertexShaderSource = R"(
    #version 330 core
    layout (location = 0) in vec3 aPos;

    uniform mat4 model;
    uniform mat4 view;
    uniform mat4 projection;

    void main() {
        gl_Position = projection * view * model * vec4(aPos, 1.0);
    }
)";

const char* volumeFragmentShaderSource = R"(
    #version 330 core
    out vec4 FragColor;

    uniform sampler3D volumeTexture;
    uniform vec3 cameraPos;

    void main() {
        vec3 rayDir = normalize(gl_FragCoord.xyz - cameraPos);
        float t = 0.0;
        vec3 texCoord = cameraPos;

        for(int i = 0; i < 200; ++i) {
            vec3 samplePos = cameraPos + rayDir * t;
            vec4 color = texture(volumeTexture, samplePos);
            color.a *= 0.1; // アルファ値を調整してボリューム感を演出

            if(color.a > 0.1) {
                FragColor = color;
                break;
            }

            t += 0.01;
        }

        if(t >= 2.0) {
            discard;
        }
    }
)";

void renderScene() {
    glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT);


    glEnable(GL_TEXTURE_3D);
    glBindTexture(GL_TEXTURE_3D, volumeTexture);


    glutSwapBuffers();
}

void initOpenGL() {
    glClearColor(0.2f, 0.2f, 0.2f, 1.0f);
    glEnable(GL_DEPTH_TEST);

    for (int i = 0; i < 3; ++i) {
        loadTexture(i);
    }

    loadVolumeTexture(); //load volume textur
}

int main(int argc, char** argv) {
    glutInit(&argc, argv);
    glutInitDisplayMode(GLUT_DOUBLE | GLUT_RGBA | GLUT_DEPTH);
    glutInitWindowSize(windowWidth, windowHeight);
    glutCreateWindow("OpenGL 3D Image Rendering");

    glewInit();
    if (glewIsSupported("GL_VERSION_2_1")) {
        std::cout << "OpenGL 2.1 is supported!" << std::endl;
    } else {
        std::cerr << "OpenGL 2.1 not supported!" << std::endl;
        return -1;
    }

    initOpenGL();

    // ボリュームレンダリング用シェーダー作成とコンパイル
    GLuint volumeVertexShader = glCreateShader(GL_VERTEX_SHADER);
    glShaderSource(volumeVertexShader, 1, &volumeVertexShaderSource, NULL);
    glCompileShader(volumeVertexShader);

    GLuint volumeFragmentShader = glCreateShader(GL_FRAGMENT_SHADER);
    glShaderSource(volumeFragmentShader, 1, &volumeFragmentShaderSource, NULL);
    glCompileShader(volumeFragmentShader);

    // シェーダープログラム作成とリンク
    GLuint volumeShaderProgram = glCreateProgram();
    glAttachShader(volumeShaderProgram, volumeVertexShader);
    glAttachShader(volumeShaderProgram, volumeFragmentShader);
    glLinkProgram(volumeShaderProgram);

    // シェーダーオブジェクト削除
    glDeleteShader(volumeVertexShader);
    glDeleteShader(volumeFragmentShader);

    // シェーダープログラム使用
    glUseProgram(volumeShaderProgram);

    // 3次元テクスチャユニフォーム設定
    glUniform1i(glGetUniformLocation(volumeShaderProgram, "volumeTexture"), 3);

    // ウィンドウループ
    while (!glfwWindowShouldClose(window)) {
        glClear(GL_COLOR_BUFFER_BIT | GL_DEPTH_BUFFER_BIT);

        // 通常のテクスチャ描画コード

        // ボリュームレンダリング
        glUseProgram(volumeShaderProgram);
        glUniform3f(glGetUniformLocation(volumeShaderProgram, "cameraPos"), cameraPos.x, cameraPos.y, cameraPos.z);
        // ボリュームテクスチャをバインド
        glBindTexture(GL_TEXTURE_3D, volumeTexture);
        // レンダリングコード

        glutSwapBuffers();
        glutPollEvents();
    }

    // プログラム終了時のクリーンアップコード

    return 0;
}